In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#importing libs
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB #this is for the model type
from sklearn import metrics
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer

In [ ]:
#reading the data
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Studily/dataset.csv")
#print(dataset.columns.tolist())
convert_dict = {'id': str,
                'title': str,
                'Category': int
                }
dataset = dataset.astype(convert_dict)
X_train_non, X_test_non, y_train, y_test = train_test_split(dataset['title'], dataset['Category'])
X_train_non = np.array(X_train_non)
X_test_non = np.array(X_test_non)
y_train = np.array(y_train)
y_test = np.array(y_test)
#X_train_non.to_csv('bye.csv')
#dataset

In [ ]:
dataset['Category'].value_counts()

1    1466
0    1247
Name: Category, dtype: int64

In [ ]:
#initialising stopwords and lematization
stopword_list = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
X_train, X_test = [], []
#pre-processing the title
for i in range(0, len(X_train_non)):
  #print(X_train_non[i])
  _title = re.sub(r'[^a-zA-Z]', ' ', X_train_non[i])
  _title = _title.lower()
  _title = _title.split()
  _title = [lemmatizer.lemmatize(word) for word in _title if not word in set(stopword_list)]
  _title = ' '.join(_title)
  X_train.append(_title)

for i in range(0, len(X_test_non)):
  _title = re.sub(r'[^a-zA-Z]', ' ', X_test_non[i])
  _title = _title.lower()
  _title = _title.split()
  _title = [lemmatizer.lemmatize(word) for word in _title if not word in set(stopword_list)]
  _title = ' '.join(_title)
  X_test.append(_title)

In [ ]:
y_test[9] #checking values of train test

1

In [ ]:
#initialising tfidf vectorizer
tf_idf = TfidfVectorizer()
tfidf_train = tf_idf.fit_transform(X_train)
tfidf_test = tf_idf.transform(X_test)

In [ ]:
#naive bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(tfidf_train, y_train)

#predicted y
y_pred = naive_bayes_classifier.predict(tfidf_test)

print(metrics.classification_report(y_test, y_pred, target_names=['Educational', 'Non-Educational']))

                 precision    recall  f1-score   support

    Educational       0.97      0.96      0.96       318
Non-Educational       0.96      0.97      0.97       361

       accuracy                           0.97       679
      macro avg       0.97      0.97      0.97       679
   weighted avg       0.97      0.97      0.97       679



In [ ]:
import pickle
filename = '/content/drive/MyDrive/Colab Notebooks/Studily/Models/Old_Data_model.sav'
pickle.dump(naive_bayes_classifier, open(filename, 'wb'))

#This next part is for doing test prediction

In [68]:
#test prediction
test_data = ['computer scientist answers computer questions from twitter']
_test = re.sub(r'[^a-zA-Z]', ' ', test_data[0])
_test = _test.lower()
_test = _test.split()
_test = [lemmatizer.lemmatize(word) for word in _test if not word in set(stopword_list)]
text_processed = [' '.join(_test)]
print(text_processed)

['computer scientist answer computer question twitter']


In [69]:
test_input = tf_idf.transform(text_processed)
test_input.shape

(1, 3846)

In [70]:
result = naive_bayes_classifier.predict(test_input[0])
if result == 1:
  print("Educational")
elif result == 0:
  print("Non-Educational")

Educational
